xml version: b37190d

In [1]:
xmlFile = 'Hexapod_Telemetry.xml'

In [2]:
import numpy as np
from astropy.time import Time
from datetime import timedelta, datetime
from lsst_efd_client import EfdClient

import pandas as pd

pd.set_option('display.max_rows', None)

In [7]:
client = EfdClient('summit_efd')
csc_index = 1

#start = Time('2020-11-03T11:41:30') #this is UTC
#end = start+timedelta(days=1)
end = Time(datetime.now())
end = Time('2020-11-06T17:20:30') #this is UTC
start = end - timedelta(seconds=5)
#import astropy
#astropy.utils.data.clear_download_cache()

In [8]:
fid = open(xmlFile)

topics = dict()
params = dict()
counts = dict()
lastT = dict()
values= dict()
nline = 0
for line in fid:
    if '<EFDB_Topic>' in line:
        i = line.find('<EFDB')
        topic = line[i+12:-14]
        t1, t2 = topic.split('_')
        dfa = await client.select_time_series('lsst.sal.%s.%s'%(t1,t2), '*', 
                                     (start-timedelta(seconds=37)).tai, (end-timedelta(seconds=37)).tai, csc_index)
    elif '<EFDB_Name>' in line:
        i = line.find('<EFDB')
        param = line[i+11:-13]
        nline+=1
        idx = '%d'%nline
    elif '<Count>' in line:
        i = line.find('<C')
        count = line[i+7:-9]
        print(topic, param,count)
        topics[idx] = topic
        params[idx] = param
        counts[idx] = count
        
        c = int(count)
        try:
            lastT[idx] = pd.to_datetime(dfa.private_sndStamp[-1], unit='s')
            if c == 1:
                values[idx] = dfa[param][-1]
            else:
                aa = [dfa['%s%d'%(param,i)][-1] for i in range(c)]
                values[idx] = np.mean(aa)
                if np.isnan(values[idx]):
                    print(aa)
        except:
            values[idx] = np.nan
    #if nline>7:
    #    break
print(nline)
fid.close()

Hexapod_Actuators Calibrated 6
Hexapod_Actuators Raw 6
Hexapod_Application Demand 6
Hexapod_Application Error 6
Hexapod_Application Position 6
Hexapod_Electrical CopleyStatusWordDrive 6
Hexapod_Electrical CopleyLatchingFaultStatus 6
7


In [9]:
frame = {'Topic': topics, 'Name': params,
         'Count': counts, 'Last Timestamp (UTC)': lastT,
         'value (average)': values}
d = pd.DataFrame(frame)

In [10]:
d

,Topic,Name,Count,Last Timestamp (UTC),value (average)
1,Hexapod_Actuators,Calibrated,6,2020-11-06 17:20:29.997608423,-1.160603e+03
2,Hexapod_Actuators,Raw,6,2020-11-06 17:20:29.997608423,3.883940e+07
3,Hexapod_Application,Demand,6,2020-11-06 17:20:29.999879837,5.533378e+02
4,Hexapod_Application,Error,6,2020-11-06 17:20:29.999879837,4.162672e-01
5,Hexapod_Application,Position,6,2020-11-06 17:20:29.999879837,5.537541e+02
6,Hexapod_Electrical,CopleyStatusWordDrive,6,2020-11-06 17:20:29.952364206,1.841000e+03
7,Hexapod_Electrical,CopleyLatchingFaultStatus,6,2020-11-06 17:20:29.952364206,0.000000e+00
